In [13]:
import json
import ssl
import bs4
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd 

print("Done")

Done


In [2]:
# SSL certificate errors might pop up
# to ignore them 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
# Request input from the user
# url = input('Enter flipkart product page, URL only')
# OR
url = "https://www.flipkart.com/mobiles/mi~brand/pr?sid=tyy,4io"

In [4]:
#Make the website believe you are accessing it from a mozilla browser
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [5]:
# Create a beautiful soup obj
soup = BeautifulSoup(webpage, 'html.parser')
html_obj = soup.prettify('utf-8')

# creating a dict to store the diff data points 
product_details = {}
product_details["highlights"] = []
product_details["product_desc"] = {}


In [6]:
# print(html_obj)

In [7]:
mainContainer = soup.find_all('div', '_1AtVbE')

In [15]:
product_details = [] #pd.DataFrame()
for block in mainContainer:
    product_details.append(block.find('div', class_='_4rR01T'))

In [16]:
product_details

[None,
 None,
 None,
 <div class="_4rR01T">Redmi 9 (Carbon Black, 64 GB)</div>,
 <div class="_4rR01T">Redmi 9 (Sky Blue, 64 GB)</div>,
 <div class="_4rR01T">Xiaomi 11Lite NE (Jazz Blue, 128 GB)</div>,
 <div class="_4rR01T">Redmi 9A (SeaBlue, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A (Midnight Black, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A (Midnight Black, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A (Nature Green, 32 GB)</div>,
 <div class="_4rR01T">Xiaomi 11Lite NE (Diamond Dazzle, 128 GB)</div>,
 <div class="_4rR01T">Redmi 9A (Nature Green, 32 GB)</div>,
 <div class="_4rR01T">Xiaomi 11Lite NE (Tuscany Coral, 128 GB)</div>,
 <div class="_4rR01T">Redmi 9A Sport (Carbon Black, 32 GB)</div>,
 <div class="_4rR01T">REDMI NOTE 10 LITE (Glacier White, 128 GB)</div>,
 <div class="_4rR01T">Redmi 9A (Sea Blue, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A Sport (Metallic Blue, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A Sport (Metallic Blue, 32 GB)</div>,
 <div class="_4rR01T">Redmi 9A 

In [17]:
# Extract from soup
# avg ratings, name, number of reviews, brand, image of product
for script in soup.findAll('script', attrs={'id': 'jsonLD'}):
    json_data = json.loads(script.text)
    product_details["ratings"] = json_data[0]["aggregateRating"]["ratingValue"]
    product_details["no_of_reviewers"] = json_data[0]["aggregateRating"]["reviewCount"]
    product_details["brand"] = json_data[0]["brand"]["name"]
    product_details["name"] = json_data[0]["name"]
    product_details["image"] = json_data[0]["image"]
    break

# Extract top highlights of the product
for li in soup.findAll('li', attrs={'class': '_2-riNZ'}):
    product_details["highlights"].append(li.text.strip())
    
# Extract the price of the product 
for div in soup.findAll('div', attrs={'class': '_1vC4DE _3qQ9m1'}):
    product_details["price_in_indian_rupees"] = float(div.text.strip()[1:].replace(",",""))
    break

# Extract key value pairs in the description of the product 
description_headers = soup.findAll('div', attrs={'class': '_2THx53'})
descriptions = soup.findAll('div', attrs={'class': '_1aK10F'})

for i, desc in enumerate(descriptions):
    product_details["product_desc"][description_headers[i].text.strip()]\
    = desc.text.strip()

# Save the product dictionary to a json file 
with open('./product_details.json', 'w', encoding="utf-8") as outfile:
    json.dump(product_details, outfile, indent=4, sort_keys=True)
    
# Save the product HTML content to a HTML file
with open('./output_file.html', 'wb') as file:
    file.write(html_obj)
    
#Finish
print("*----Extraction of Data is Complete. Check json file----*")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)